# Podcast Episode Erstellen

### imports

In [ ]:
import sys
sys.path.append(".")
from audio_transcription.faster_whisper_word_level import transcribe
from Episodes_Downloader.episodes_downloader import get_names_and_urls_all_episodes, download_and_save_mp3_in_dir
import os
import pandas as pd
from pydub import AudioSegment
from pydub.playback import play
from playsound import playsound
import IPython 
import io
from db_connect import db_get_df,db_save_df
from segment_ranking.rank_segments import get_most_similar_documents_Llama2
from Audio_segmentation.split_audio import produce_snippets
from Audio_segmentation.concat_audio import produce_audio
from tqdm import tqdm

### Episoden URLs laden

In [ ]:
titles, audio_urls = get_names_and_urls_all_episodes()
print(titles)

In [ ]:
print(len(titles))

In [ ]:
df = pd.DataFrame({"title": titles, "download_url": audio_urls, "transcript": None})
df.dtypes

In [ ]:
db_save_df(df, "transcripts_all")

### Alle mp3s herunterladen

In [ ]:
path = "/Volumes/Samsung_T5/Podcast_Episoden"
os.listdir(path)

In [ ]:
df = db_get_df("transcripts_all")

for i, row in tqdm(df.iterrows()):
    title = row["title"].replace("/","")
    if title in os.listdir(path):
        continue
    download_and_save_mp3_in_dir(row["download_url"], path, title)


In [ ]:
df.loc[179, "download_url"]

### Transkription auf Word level Ebene

In [ ]:
df = transcribe("/Users/br/Projects/Bachelorarbeit/data/Episode_audio_files/jonathan swift gullivers reisen 2.mp3")
# db_save_df(df,"transcript_word_level")

In [ ]:
db_save_df(df, "transcript_gulliver_word_level")

### Debug db

In [ ]:
import sqlite3
import pandas as pd

con = sqlite3.connect("/Volumes/Samsung_T5/Backup/transcripts.sqlite")
df = pd.read_sql_query(f"SELECT * FROM transcript_word_level_2237", con)
con.close()

In [ ]:
print(df.dtypes)
print(df["filename"].drop_duplicates(ignore_index=True))

Die Namen der Filenames ändern

In [ ]:
df['filename'] = df['filename'].str.replace('/nfs/scratch/students/neumannvi84434/Podcast_Episoden/', '')

In [ ]:
with sqlite3.connect("/Volumes/Samsung_T5/Backup/transcripts.sqlite") as con:
    df.to_sql("transcript_word_level_2237", con, index=False, if_exists='replace')

### Satzbildung mit spacy

In [ ]:
import spacy

In [ ]:
df = db_get_df("transcript_gulliver_word_level")
df.dtypes

Zunächst erstmal mit Whispers Punkten

In [ ]:
print(f'|{df["word"][2]}|')

In [ ]:
sentences = []
one_sentence = []
for word in df["word"]:
    one_sentence.append(word.strip())
    if "." in word:
        sentences.append(" ".join(one_sentence))
        one_sentence = []
print(sentences)

### Demonstration

In [ ]:
df = db_get_df(table="transcripts")

In [ ]:
userInputText = "Auf dem l Feiern und trinken"
userInputTime = 7
best_fitting = get_most_similar_documents_Llama2(userInputText, userInputTime)
print(best_fitting["segment_text"].to_markdown())

produce_snippets()
produce_audio()


In [ ]:
IPython.display.Audio("concatenated_audio.mp3")